In [ ]:
#dependencies
import pandas as pd 

#loading the data
df = pd.read_csv("../input/HR_comma_sep.csv")

#checking for NULL values
print("\033[1m" + "\033[94m" + "Data types:\n" + 11 * "-")
print("\033[30m" + "{}\n".format(df.dtypes))
print("\033[1m" + "\033[94m" + "Sum of null values in each column:\n" + 35 * "-")
print("\033[30m" + "{}".format(df.isnull().sum()))
df.head()


Data preprocessing

1: Convert salary values into integers 

In [ ]:
# Map salary into integers
salary_map = {"low": 0, "medium": 1, "high": 2}
df["salary"] = df["salary"].map(salary_map)

In [ ]:
df = pd.get_dummies(df, columns=["Department"], drop_first=True)

**The data now has 17 features**

In [ ]:
#converting dataframe into numpy array (80/20 split)
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np

x = df.loc[:, df.columns != "left"].values
y = df.loc[:, df.columns == "left"].values.flatten()

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = 1)

#upsampling minority classes 
x_train_u, y_train_u = resample(x_train[y_train == 1], y_train[y_train == 1], replace=True, n_samples = x_train[y_train == 0].shape[0],
                                random_state=1)

x_train_u = np.concatenate((x_train[y_train == 0], x_train_u))
y_train_u = np.concatenate((y_train[y_train == 0], y_train_u))

# Downsampling majority class
x_train_d, y_train_d = resample(x_train[y_train == 0], y_train[y_train == 0], replace=True, n_samples= x_train[y_train == 1].shape[0],
                                random_state=1)

x_train_d = np.concatenate((x_train[y_train == 1], x_train_d))
y_train_d = np.concatenate((y_train[y_train == 1], y_train_d))

Machine Learning algorithm used: Random Forest

Hyperparameters tuned: max_features and min_samples_leaf

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=50, class_weight="balanced", random_state=123, 
                            criterion="entropy", max_features = 0.4, min_samples_leaf = 1)
clf.fit(x_train_u, y_train_u)


In [ ]:
from sklearn.metrics import accuracy_score

print('Random Forest Accuracy: {:.3f}'.format(accuracy_score(y_test, clf.predict(x_test))))